# EDA On Ashrae Energy Consumption Dataset 

Analyzing an interesting dataset found on kaggle from ASHARE which stands for American Society of Heating, Refrigeratinga nd Air-Conditioning Engineering

### How to run the code

This is an executable [*Jupyter notebook*](https://jupyter.org) hosted on [Jovian.ml](https://www.jovian.ml), a platform for sharing data science projects. You can run and experiment with the code in a couple of ways: *using free online resources* (recommended) or *on your own computer*.

#### Option 1: Running using free online resources (1-click, recommended)

The easiest way to start executing this notebook is to click the "Run" button at the top of this page, and select "Run on Binder". This will run the notebook on [mybinder.org](https://mybinder.org), a free online service for running Jupyter notebooks. You can also select "Run on Colab" or "Run on Kaggle".


#### Option 2: Running on your computer locally

1. Install Conda by [following these instructions](https://conda.io/projects/conda/en/latest/user-guide/install/index.html). Add Conda binaries to your system `PATH`, so you can use the `conda` command on your terminal.

2. Create a Conda environment and install the required libraries by running these commands on the terminal:

```
conda create -n zerotopandas -y python=3.8 
conda activate zerotopandas
pip install jovian jupyter numpy pandas matplotlib seaborn opendatasets --upgrade
```

3. Press the "Clone" button above to copy the command for downloading the notebook, and run it on the terminal. This will create a new directory and download the notebook. The command will look something like this:

```
jovian clone notebook-owner/notebook-id
```



4. Enter the newly created directory using `cd directory-name` and start the Jupyter notebook.

```
jupyter notebook
```

You can now access Jupyter's web interface by clicking the link that shows up on the terminal or by visiting http://localhost:8888 on your browser. Click on the notebook file (it has a `.ipynb` extension) to open it.


## Downloading the Dataset

We utilize the power of the OpenDatasets library to download the dataset from Kaggle. By leveraging this library, we can easily access and acquire the necessary dataset for our analysis

In [6]:
!pip install humanize --quiet


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install opendatasets --upgrade --quiet


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Let's begin by downloading the data, and listing the files within the dataset.

In [2]:

dataset_url = 'https://www.kaggle.com/competitions/ashrae-energy-prediction/data'

In [ ]:
import opendatasets as od
od.download(dataset_url) #run this to download the dataset

The dataset has been downloaded and extracted.

In [19]:
# Change this
data_dir = './ashrae-energy-prediction'

In [20]:
import os
os.listdir(data_dir)

['building_metadata.csv',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 'weather_test.csv',
 'weather_train.csv']

## Data Description

As obsereved there are 6 different csv files in the downloaded zip file 

The given data description in the kaggle website:

train.csv
- building_id - Foreign key for the building metadata.
- meter - The meter id code. Read as {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}. Not every building has all meter types.
- timestamp - When the measurement was taken
- meter_reading - The target variable. Energy consumption in kWh (or equivalent). Note that this is real data with measurement error, which we expect will impose a baseline level of modeling error. UPDATE: as discussed here, the site 0 electric meter readings are in kBTU.

building_meta.csv
- site_id - Foreign key for the weather files.
- building_id - Foreign key for training.csv
- primary_use - Indicator of the primary category of activities for the building based on EnergyStar property type definitions
- square_feet - Gross floor area of the building
- year_built - Year building was opened
- floor_count - Number of floors of the building

weather_[train/test].csv
Weather data from a meteorological station as close as possible to the site.

- site_id
- air_temperature - Degrees Celsius
- timestamp - When the measurement was taken
- cloud_coverage - Portion of the sky covered in clouds, in oktas
- dew_temperature - Degrees Celsius
- precip_depth_1_hr - Millimeters
- sea_level_pressure - Millibar/hectopascals
- wind_direction - Compass direction (0-360)
- wind_speed - Meters per second

Let us save and upload our work to Jovian before continuing.

In [13]:
project_name = "EDA_Ashrae_Energy_consumption" # change this (use lowercase letters and hyphens only)

In [14]:
!pip install jovian --upgrade -q


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import jovian

<IPython.core.display.Javascript object>

In [ ]:
jovian.commit(project=project_name)

## Loading the CSV using Pandas

Lets use pandas library to load only the required unprocesssed data into different variables for our analysis

In [8]:
import pandas as pd

# Read the train.csv file
train_df = pd.read_csv('./ashrae-energy-prediction/train.csv',parse_dates=['timestamp'])

weather_train_df= pd.read_csv('./ashrae-energy-prediction/weather_train.csv',parse_dates=['timestamp'])

meta_df = pd.read_csv('./ashrae-energy-prediction/building_metadata.csv',parse_dates=['year_built'])

The parse_dates parameter in the pd.read_csv() function is used to specify which columns should be parsed as dates during the reading of the CSV file. 

Selected the columns that must be parsed as dates by going through the data descrption

## Exploring the Data Frame 

Lets take a look at dffierent columns in different csv's 

In [25]:
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01,0.0
1,1,0,2016-01-01,0.0
2,2,0,2016-01-01,0.0
3,3,0,2016-01-01,0.0
4,4,0,2016-01-01,0.0


In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   building_id    int64         
 1   meter          int64         
 2   timestamp      datetime64[ns]
 3   meter_reading  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 616.9 MB


In [28]:
for column in train_df.columns:
    unique_values = train_df[column].nunique()
    print(f"Unique values in column '{column}': {unique_values}")


Unique values in column 'building_id': 1449
Unique values in column 'meter': 4
Unique values in column 'timestamp': 8784
Unique values in column 'meter_reading': 1688175


In [24]:
train_df.describe()

,building_id,meter,timestamp,meter_reading
count,2.021610e+07,2.021610e+07,20216100,2.021610e+07
mean,7.992780e+02,6.624412e-01,2016-07-03 22:59:40.301601792,2.117121e+03
min,0.000000e+00,0.000000e+00,2016-01-01 00:00:00,0.000000e+00
25%,3.930000e+02,0.000000e+00,2016-04-05 21:00:00,1.830000e+01
50%,8.950000e+02,0.000000e+00,2016-07-04 17:00:00,7.877500e+01
75%,1.179000e+03,1.000000e+00,2016-10-02 22:00:00,2.679840e+02
max,1.448000e+03,3.000000e+00,2016-12-31 23:00:00,2.190470e+07
std,4.269133e+02,9.309921e-01,NaN,1.532356e+05


**train_df**
After taking a look at the train_df dataset we can can observe the following things :

- There are 4 columns in the dataset where the building_id is int64 datatype and has 1149 unique values
    -  Where the building_id is int64 datatype and has 1149 unique values (**foreign key** for meta_df) given in the dataset description
    -  The meter column is also of int64 datatype and has 4 unique values
    -  The timestamp column is of datetime64[ns]
    - Similarly the meter_reading column is of float64.


Now similarly lets check for the weather_df column too:


In [29]:
weather_train_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [36]:
weather_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   site_id             139773 non-null  int64         
 1   timestamp           139773 non-null  datetime64[ns]
 2   air_temperature     139718 non-null  float64       
 3   cloud_coverage      70600 non-null   float64       
 4   dew_temperature     139660 non-null  float64       
 5   precip_depth_1_hr   89484 non-null   float64       
 6   sea_level_pressure  129155 non-null  float64       
 7   wind_direction      133505 non-null  float64       
 8   wind_speed          139469 non-null  float64       
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 9.6 MB


In [31]:
for column in weather_train_df.columns:
    unique_values = weather_train_df[column].nunique()
    print(f"Unique values in column '{column}': {unique_values}")

Unique values in column 'site_id': 16
Unique values in column 'timestamp': 8784
Unique values in column 'air_temperature': 619
Unique values in column 'cloud_coverage': 10
Unique values in column 'dew_temperature': 522
Unique values in column 'precip_depth_1_hr': 128
Unique values in column 'sea_level_pressure': 709
Unique values in column 'wind_direction': 43
Unique values in column 'wind_speed': 58


In [34]:
weather_train_df.describe()

,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
count,139773.000000,139718.000000,70600.000000,139660.000000,89484.000000,129155.000000,133505.000000,139469.000000
mean,7.478977,14.418106,2.149306,7.350158,0.983047,1016.158038,180.526632,3.560527
std,4.604744,10.626595,2.599150,9.790235,8.463678,7.629684,111.523629,2.335874
min,0.000000,-28.900000,0.000000,-35.000000,-1.000000,968.200000,0.000000,0.000000
25%,3.000000,7.200000,0.000000,0.600000,0.000000,1011.800000,80.000000,2.100000
50%,7.000000,15.000000,2.000000,8.300000,0.000000,1016.400000,190.000000,3.100000
75%,11.000000,22.200000,4.000000,14.400000,0.000000,1020.800000,280.000000,5.000000
max,15.000000,47.200000,9.000000,26.100000,343.000000,1045.500000,360.000000,19.000000


**weather_train_df**

Similarly there are 8 columns in the weather_train_df with 139772 rows 

- The site_id has 16 unique values and is of int 64
- The air_temperature, cloud_coverage, dew_temperature, precip_depth_1_hr, sea_level_pressure, wind_direction and wind_speed is of float64
- And the timestamp is of datetime64[ns]


And finally lets take a look at meta_df

In [37]:
meta_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008-01-01,NaN
1,0,1,Education,2720,2004-01-01,NaN
2,0,2,Education,5376,1991-01-01,NaN
3,0,3,Education,23685,2002-01-01,NaN
4,0,4,Education,116607,1975-01-01,NaN


In [38]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1449 entries, 0 to 1448
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   site_id      1449 non-null   int64         
 1   building_id  1449 non-null   int64         
 2   primary_use  1449 non-null   object        
 3   square_feet  1449 non-null   int64         
 4   year_built   675 non-null    datetime64[ns]
 5   floor_count  355 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 68.0+ KB


In [42]:
meta_df.sample(3)

,site_id,building_id,primary_use,square_feet,year_built,floor_count
707,5,707,Education,11270,1976-01-01,1.0
946,9,946,Education,177216,NaT,NaN
903,9,903,Education,275793,NaT,NaN


In [43]:
meta_df['primary_use'].nunique()

16

In [41]:
meta_df.describe()

,site_id,building_id,square_feet,year_built,floor_count
count,1449.000000,1449.000000,1449.000000,675,355.000000
mean,6.952381,724.000000,92111.776398,1967-12-16 15:40:48,3.740845
min,0.000000,0.000000,283.000000,1900-01-01 00:00:00,1.000000
25%,3.000000,362.000000,23012.000000,1949-01-01 00:00:00,1.000000
50%,5.000000,724.000000,57673.000000,1970-01-01 00:00:00,3.000000
75%,13.000000,1086.000000,115676.000000,1995-01-01 00:00:00,5.000000
max,15.000000,1448.000000,875000.000000,2017-01-01 00:00:00,26.000000
std,5.003432,418.434583,110769.950997,NaN,3.333683


**meta_df**

This is dataset consists of 6 columns with 1448 rows 

- The site_id, building_id and square_feet is of int64 datatype  (site_id is a **foreign key** for weather_train_df) given in the dataset description
- The primary use is made up of 16 categories
- year_built is of float64 and timestamp just like in other dataset is of datetime64[ns]

## Data Preparation and Cleaning

**Data Preperation** 

Lets change the datatypes of columns so it would reduce the memory usage.

Before reducing the datatypes lets check the overall memory usage of every single dataframe


In [10]:
from humanize import naturalsize

memory_usage = [naturalsize(df.memory_usage().sum()) for df in [train_df, weather_train_df, meta_df]]
print(memory_usage)

['646.9 MB', '10.1 MB', '69.7 kB']


Thats quite high lets reduce the datatypes and then see how much of a difference does it make.

In [3]:
train_df=train_df.astype({'building_id':'int16','meter':'int8','meter_reading':'float32'})

In [4]:
weather_train_df=weather_train_df.astype({'site_id':'int8','air_temperature':'float16',
                       'cloud_coverage':'float16','dew_temperature':'float16',
                       'precip_depth_1_hr':'float16','sea_level_pressure':'float16'
                       ,'wind_speed':'float16','wind_direction':'float16'
                       })

In [5]:
meta_df=meta_df.astype({'site_id':'int8','building_id':'int16',
                       'square_feet':'int16'
                       })

In [7]:
from humanize import naturalsize

memory_usage = [naturalsize(df.memory_usage().sum()) for df in [train_df, weather_train_df, meta_df]]
print(memory_usage)


['303.2 MB', '3.2 MB', '42.1 kB']


That has significantly reduced. Great! 

## Exploratory Analysis

Lets Plot differnt graphs using each data frame and see what visualze what correlation does it have between one another!!

In [11]:
train_df.corr()

,building_id,meter,timestamp,meter_reading
building_id,1.000000,0.222268,0.014435,0.008761
meter,0.222268,1.000000,0.005264,0.017672
timestamp,0.014435,0.005264,1.000000,-0.006679
meter_reading,0.008761,0.017672,-0.006679,1.000000


In [15]:
train_df.isnull().sum() #checking for missing values in the dataset 

building_id      0
meter            0
timestamp        0
meter_reading    0
dtype: int64

In [12]:
weather_train_df.corr()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
site_id,1.000000,0.001924,-0.262746,0.021003,-0.204501,0.020269,-0.009193,0.045021,0.027688
timestamp,0.001924,1.000000,0.184521,-0.017758,0.240282,0.003878,0.131250,-0.029262,-0.081549
air_temperature,-0.262746,0.184521,1.000000,0.010149,0.790386,-0.010719,-0.209317,-0.093240,-0.056743
cloud_coverage,0.021003,-0.017758,0.010149,1.000000,0.183606,0.122253,-0.158361,0.052560,0.294019
dew_temperature,-0.204501,0.240282,0.790386,0.183606,1.000000,0.065881,-0.168759,-0.149033,-0.096836
precip_depth_1_hr,0.020269,0.003878,-0.010719,0.122253,0.065881,1.000000,-0.081343,-0.013161,0.035023
sea_level_pressure,-0.009193,0.131250,-0.209317,-0.158361,-0.168759,-0.081343,1.000000,-0.103002,-0.251602
wind_direction,0.045021,-0.029262,-0.093240,0.052560,-0.149033,-0.013161,-0.103002,1.000000,0.354690
wind_speed,0.027688,-0.081549,-0.056743,0.294019,-0.096836,0.035023,-0.251602,0.354690,1.000000


In [22]:
import jovian

In [23]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'

## Exploratory Analysis and Visualization

**TODO** - write some explanation here.



> Instructions (delete this cell)
> 
> - Compute the mean, sum, range and other interesting statistics for numeric columns
> - Explore distributions of numeric columns using histograms etc.
> - Explore relationship between columns using scatter plots, bar charts etc.
> - Make a note of interesting insights from the exploratory analysis

Let's begin by importing`matplotlib.pyplot` and `seaborn`.

In [24]:
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (9, 5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

**TODO** - Explore one or more columns by plotting a graph below, and add some explanation about it

Let us save and upload our work to Jovian before continuing

In [25]:
import jovian

In [26]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'

## Asking and Answering Questions

TODO - write some explanation here.



> Instructions (delete this cell)
>
> - Ask at least 5 interesting questions about your dataset
> - Answer the questions either by computing the results using Numpy/Pandas or by plotting graphs using Matplotlib/Seaborn
> - Create new columns, merge multiple dataset and perform grouping/aggregation wherever necessary
> - Wherever you're using a library function from Pandas/Numpy/Matplotlib etc. explain briefly what it does



#### Q1: TODO - ask a question here and answer it below

#### Q2: TODO - ask a question here and answer it below

#### Q3: TODO - ask a question here and answer it below

#### Q4: TODO - ask a question here and answer it below

#### Q5: TODO - ask a question here and answer it below

Let us save and upload our work to Jovian before continuing.

In [28]:
import jovian

In [29]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'

## Inferences and Conclusion

**TODO** - Write some explanation here: a summary of all the inferences drawn from the analysis, and any conclusions you may have drawn by answering various questions.

In [30]:
import jovian

In [31]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'

## References and Future Work

**TODO** - Write some explanation here: ideas for future projects using this dataset, and links to resources you found useful.

> Submission Instructions (delete this cell)
> 
> - Upload your notebook to your Jovian.ml profile using `jovian.commit`.
> - **Make a submission here**: https://jovian.ml/learn/data-analysis-with-python-zero-to-pandas/assignment/course-project
> - Share your work on the forum: https://jovian.ml/forum/t/course-project-on-exploratory-data-analysis-discuss-and-share-your-work/11684
> - Share your work on social media (Twitter, LinkedIn, Telegram etc.) and tag [@JovianML](https://twitter.com/jovianml)
>
> (Optional) Write a blog post
> 
> - A blog post is a great way to present and showcase your work.  
> - Sign up on [Medium.com](https://medium.com) to write a blog post for your project.
> - Copy over the explanations from your Jupyter notebook into your blog post, and [embed code cells & outputs](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e)
> - Check out the Jovian.ml Medium publication for inspiration: https://medium.com/jovianml


 

In [32]:
import jovian

In [35]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'